# TP description d'image avec un Transformer
Ce TP est la suite du TP portant sur l'implémentation d'un RNN. Dans ce TP, le RNN sera remplacé par un Transformer dans le but d'obtenir un système de description d'image, c'est-à-dire un système capable de générer une phrase décrivant une image.    
1. Lancer une session linux (et non pas windows)
2. Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
3. Dans ce terminal, taper la commande suivante pour lancer Spyder : `spyder &`
4. Configurer Spyder en suivant ces instructions : [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
5. Créer un dossier `TP_description_image_Transformer` et placer le dossier `utils` à l'intérieur.
6. Créer un script python `tp.py` dans le dossier `TP_description_image_Transformer` et coller les lignes de code suivantes : 

In [1]:
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
from utils.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from utils.image_utils import image_from_url
from utils.transformer_layers import *
from utils.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from utils.transformer import CaptioningTransformer
from utils.captioning_solver_transformer import CaptioningSolverTransformer


plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Base de données Microsoft COCO
Comme lors du TP RNN, nous utiliserons la base de données [Microsoft COCO](http://mscoco.org/).
Attention : la base de données est d'environ ~256Mo. Il faudra donc la télécharger et la décompresser dans `/tmp`. La base de données est disponible sur Thor, dans l'onglet "Documents".

Les données ont été pré-traitées. Pour chaque image, un vecteur (de taille 4096) issu de la couche fc7 d'un VGG-16 (pré-entraîné sur ImageNet) a été extrait, puis réduit à une taille de 512 (avec une ACP) avant d'être stocké dans les fichiers `train2014_vgg16_fc7_pca.h5` et `val2014_vgg16_fc7_pca.h5`.

Charger la base de données à l'aide des lignes de code suivantes :

In [ ]:
data = load_coco_data()

# Print out all the keys and values from the data dictionary
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

## Visualiser les données
De manière générale, il est indispensable de regarder les données que nous nous apprêtons à traiter.

Vous pouvez visualiser un minibatch de la manière suivante :

In [ ]:
# Sample a minibatch and show the images and captions
batch_size = 3

captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)
for i, (caption, url) in enumerate(zip(captions, urls)):
    im_cur = image_from_url(url)
    if(type(im_cur)!=type(None)):# if the image was downloaded
        plt.imshow(im_cur)
        plt.axis('off')
        caption_str = decode_captions(caption, data['idx_to_word'])
        plt.title(caption_str)
        plt.show()

# Transformer
Nous utiliserons un Transformer pour réaliser la description d'image. Le fichier `utils/transformer_layers.py` contient les implémentations de différentes couches permettant d'obtenir un Transformer. Le fichier `utils/transformer.py` utilise les couches définies dans le fichier précedent pour implémenter le modèle de description d'image.

Nous débuterons avec l'implémentation des couches dans le fichier `utils/transformer_layers.py`.

# Transformer: Attention softmax à tes têtes multiples

### Attention softmax ("Dot-product attention")

Comme vu en cours, une opération d'attention ("cross-attention") entre un vecteur "query" $q\in\mathbb{R}^d$, un semble de vecteurs "value" $\{v_1,\dots,v_n\}, v_i\in\mathbb{R}^d$, et un ensemble de vecteurs "key" $\{k_1,\dots,k_n\}, k_i \in \mathbb{R}^d$ :

\begin{align}
c = \sum_{i=1}^{n} v_i \alpha_i 
\end{align}
où
\begin{align}
\alpha_i = \frac{\exp(k_i^\top q)}{\sum_{j=1}^{n} \exp(k_j^\top q)}. \\
\end{align}

Les $\alpha_i$ sont appelés les scores d'attention ou les poids d'attention. frequently called the "attention weights". Le vecteur de sortie $c\in\mathbb{R}^d$ est une combinaison linéaire des vecteurs "value".

### Couche d'inter-attention ("Cross-attention")
Dans le cas d'une couche d'inter-attention, les vecteurs "value", "key" proviennent d'une matrice $Y \in \mathbb{R}^{\ell_y \times d}$, où $\ell_y$ est la longueur de la séquence des $y_i$. Les vecteurs "query" proviennent d'une matrice $X \in \mathbb{R}^{\ell_x \times d}$, où $\ell_x$ est la longueur de la séquence des $x_i$. Ainsi, les paramètres optimisés sont les matrices $V,K,Q \in \mathbb{R}^{d\times d}$ qui transforment $X$  et $Y$ de la manière suivante :

\begin{align}
v_i = Vy_i\ \ i \in \{1,\dots,\ell_y\}\\
k_i = Ky_i\ \ i \in \{1,\dots,\ell_y\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell_x\}
\end{align}

### Couche d'auto-attention ("Self-attention")
Dans le cas d'une couche d'auto-attention, les vecteurs "value", "key" and "query" proviennent de l'entrée $X \in \mathbb{R}^{\ell \times d}$, où $\ell$ est la longueur de la séquence. Ainsi, les paramètres optimisés sont les matrices $V,K,Q \in \mathbb{R}^{d\times d}$ qui transforment l'entrée $X$ de la manière suivante :

\begin{align}
v_i = Vx_i\ \ i \in \{1,\dots,\ell\}\\
k_i = Kx_i\ \ i \in \{1,\dots,\ell\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell\}
\end{align}

### Attention softmax à têtes multiples ("Multi-Headed Scaled Dot-Product Attention")
Dans le cas de l'attention softmax à têtes multiples, les matrices $V,K,Q$ diffèrent pour chaque tête. Ainsi, le modèle gagne en expressivité en permettant d'apprendre "à attirer l'attention" de manières différentes. Appelons $h$ le nombre de têtes, $Z_i$ la sortie de la tête $i$ et $Q_i$, $K_i$ et $V_i$ les matrices optimisées. Afin de conserver un coût calculatoire proche du cas à une seule tête, nous choisirons $Q_i \in \mathbb{R}^{d\times d/h}$, $K_i \in \mathbb{R}^{d\times d/h}$ et $V_i \in \mathbb{R}^{d\times d/h}$. En ajoutant un terme de normalisation $\frac{1}{\sqrt{d/h}}$, nous obtenons :

\begin{equation}
Z_i = \text{softmax}\bigg(\frac{(X Q_i)(Y K_i)^\top}{\sqrt{d/h}}\bigg)(Y V_i)
\end{equation}

où $Z_i\in\mathbb{R}^{\ell \times d/h}$ et $\ell_x$ est de la longueur de la séquence des $x_i$.

Dans cette implémentation, nous utiliserons une couche de dropout aux scores d'attention:

\begin{equation}
Z_i = \text{dropout}\bigg(\text{softmax}\bigg(\frac{(XQ_i)(YK_i)^\top}{\sqrt{d/h}}\bigg)\bigg)(YV_i)
\end{equation}

Finallement, une dernière transformation affine est appliquée sur chaque $Z_i$ avant de la sommer, ce qui peut s'écrire comme une transformation affine appliquée sur la concaténation des $Z_i$:

\begin{equation}
Z = [Z_1;\dots;Z_h]A
\end{equation}

où $A \in\mathbb{R}^{d\times d}$ et $[Z_1;\dots;Z_h]\in\mathbb{R}^{\ell \times d}$.

Ouvrir le fichier `utils/transformer_layers.py`.

**À Coder :** compléter la classe `MultiHeadAttention`. Vous pouvez tester votre code en exécutant le morceau de code suivant (vous devriez obtenir une erreur faible, de l'ordre de e-3) :

In [ ]:
torch.manual_seed(231)

# Choose dimensions such that they are all unique for easier debugging:
# Specifically, the following values correspond to N=1, H=2, T=3, E//H=4, and E=8.
batch_size = 1
sequence_length = 3
embed_dim = 8
attn = MultiHeadAttention(embed_dim, num_heads=2)

# Self-attention.
data = torch.randn(batch_size, sequence_length, embed_dim)
self_attn_output = attn(query=data, key=data, value=data)

# Masked self-attention.
mask = torch.randn(sequence_length, sequence_length) < 0.5
masked_self_attn_output = attn(query=data, key=data, value=data, attn_mask=mask)

# Attention using two inputs.
other_data = torch.randn(batch_size, sequence_length, embed_dim)
attn_output = attn(query=data, key=other_data, value=other_data)

expected_self_attn_output = np.asarray([[
[-0.2494,  0.1396,  0.4323, -0.2411, -0.1547,  0.2329, -0.1936,
          -0.1444],
         [-0.1997,  0.1746,  0.7377, -0.3549, -0.2657,  0.2693, -0.2541,
          -0.2476],
         [-0.0625,  0.1503,  0.7572, -0.3974, -0.1681,  0.2168, -0.2478,
          -0.3038]]])

expected_masked_self_attn_output = np.asarray([[
[-0.1347,  0.1934,  0.8628, -0.4903, -0.2614,  0.2798, -0.2586,
          -0.3019],
         [-0.1013,  0.3111,  0.5783, -0.3248, -0.3842,  0.1482, -0.3628,
          -0.1496],
         [-0.2071,  0.1669,  0.7097, -0.3152, -0.3136,  0.2520, -0.2774,
          -0.2208]]])

expected_attn_output = np.asarray([[
[-0.1980,  0.4083,  0.1968, -0.3477,  0.0321,  0.4258, -0.8972,
          -0.2744],
         [-0.1603,  0.4155,  0.2295, -0.3485, -0.0341,  0.3929, -0.8248,
          -0.2767],
         [-0.0908,  0.4113,  0.3017, -0.3539, -0.1020,  0.3784, -0.7189,
          -0.2912]]])

print('self_attn_output error: ', rel_error(expected_self_attn_output, self_attn_output.detach().numpy()))
print('masked_self_attn_output error: ', rel_error(expected_masked_self_attn_output, masked_self_attn_output.detach().numpy()))
print('attn_output error: ', rel_error(expected_attn_output, attn_output.detach().numpy()))

# Encodage de la position ("Positional encoding")

Comme vu en cours, l'encodage de la position est très important. Ici la technique utilisée est décrite par les équations suivantes:

$P \in \mathbb{R}^{l\times d}$, où $P_{ij} = $

$$
\begin{cases}
\text{sin}\left(i \cdot 10000^{-\frac{j}{d}}\right) & \text{si j est pair} \\
\text{cos}\left(i \cdot 10000^{-\frac{(j-1)}{d}}\right) & \text{sinon} \\
\end{cases}
$$

Cet encodage de la position est ajouté à la matrice $X$ avant d'entrer dans la couceh d'attention : $X + P$.

**À Coder :** compléter la classe `PositionalEncoding`. Vous pouvez tester votre code en exécutant le morceau de code suivant (vous devriez obtenir une erreur faible, de l'ordre de e-3).

In [ ]:
torch.manual_seed(231)

batch_size = 1
sequence_length = 2
embed_dim = 6
data = torch.randn(batch_size, sequence_length, embed_dim)

pos_encoder = PositionalEncoding(embed_dim)
output = pos_encoder(data)

expected_pe_output = np.asarray([[[-1.2340,  1.1127,  1.6978, -0.0865, -0.0000,  1.2728],
                                  [ 0.9028, -0.4781,  0.5535,  0.8133,  1.2644,  1.7034]]])

print('pe_output error: ', rel_error(expected_pe_output, output.detach().numpy()))

# Questions

Lors des étapes précedentes, plusieurs choix ont été effectués. Expliquer les raisons des choix suivants :
1. Utiliser plusieurs têtes plutôt qu'une seule
2. Normaliser le produit scalaire par $\sqrt{d/h}$. Rappelons ici que $d$ est la dimension des vecteurs et $h$ est le nombre de têtes.
3. Ajouter une transformation linéaire en sortie.

# Transformer pour la description d'une image
Maintenant que l'implémentation d'une couche d'attention est effectuée, nous pouvons les utiliser pour obtenir un système de description d'image utilisant un Transformer.

**À Coder :**  Ouvrir le fichier `utils/transformer.py`. Implémenter la fonction `forward` de la classe `CaptioningTransformer`.

Vous pouvez tester votre code en exécutant le morceau de code suivant (vous devriez obtenir une erreur faible, de l'ordre de e-5).

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

N, D, W = 4, 20, 30
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 3

transformer = CaptioningTransformer(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    num_heads=2,
    num_layers=2,
    max_length=30
)

# Set all model parameters to fixed values
for p in transformer.parameters():
    p.data = torch.tensor(np.linspace(-1.4, 1.3, num=p.numel()).reshape(*p.shape))

features = torch.tensor(np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D))
captions = torch.tensor((np.arange(N * T) % V).reshape(N, T))

scores = transformer(features, captions)
expected_scores = np.asarray([[[-16.9532,   4.8261,  26.6054],
         [-17.1033,   4.6906,  26.4844],
         [-15.0708,   4.1108,  23.2924]],
        [[-17.1767,   4.5897,  26.3562],
         [-15.6017,   4.8693,  25.3403],
         [-15.1028,   4.6905,  24.4839]],
        [[-17.2172,   4.7701,  26.7574],
         [-16.6755,   4.8500,  26.3754],
         [-17.2172,   4.7701,  26.7574]],
        [[-16.3669,   4.1602,  24.6872],
         [-16.7897,   4.3467,  25.4831],
         [-17.0103,   4.7775,  26.5652]]])
print('scores error: ', rel_error(expected_scores, scores.detach().numpy()))

# Lancer un apprentissage sur une petite base de données
exécuter le code suivant pour lancer un apprentissage (on devrait plutôt parler de sur-apprentissage) sur une base de données de 50 couples image/phrase. Le coût final devrait être inférieur à 0.03.

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

data = load_coco_data(max_train=50)

transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          wordvec_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30
        )


transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=100,
           batch_size=25,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

print('Final loss: ', transformer_solver.loss_history[-1])

# Génération d'une description d'une image ("Test-time")
La fonction permettant de générer une description a déjà été implémentée, ainsi vous pouvez générer des descriptions (en utilisant le modèle précedemment sur-entraîné) sur la base d'apprentissage ainsi que sur la base de validation, en exécutant le code ci-après. Vous devriez constater que les descriptions obtenues pour les exemples de la base d'apprentissage sont très satisfaisants (car le Tranformer a sur-appris sur cette base). En revanche, les résultats obtenus pour les exemples de la base de validation n'auront probablement aucun sens.

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)            
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()